# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
citiescsv = "weatherpy-d-pranjic.csv"
data = pd.read_csv(citiescsv)
data = data.drop(columns=['Unnamed: 0'])
data

,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Yārāda,IN,1630791430,17.6500,83.2667,79.12,100,75,5.75
1,Grindavik,IS,1630791441,63.8424,-22.4338,55.35,100,90,36.82
2,Hambantota,LK,1630791411,6.1241,81.1185,77.34,100,40,4.61
3,Sandakan,MY,1630791552,5.8402,118.1179,74.88,100,20,2.30
4,Petropavlovsk-Kamchatskiy,RU,1630791528,53.0452,158.6483,50.59,100,0,6.71
...,...,...,...,...,...,...,...,...,...
575,Aranos,NaN,1630791425,-24.1333,19.1167,62.46,15,31,9.57
576,Uyuni,BO,1630791452,-20.4597,-66.8250,56.08,14,2,6.24
577,Ada,US,1630791555,43.4166,-116.1262,87.73,11,1,4.61
578,West Wendover,US,1630791249,40.7391,-114.0733,83.41,9,1,9.22


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


locations = data[["Latitude", "Longitude"]]

humidity = data["Humidity"].astype(float)

In [4]:


mapa = gmaps.figure()


vlaga = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)


mapa.add_layer(vlaga)

mapa


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacayspots1 = data.loc[(data['Max Temperature']<=85) 
                       & (data['Max Temperature']>69)]
vacayspots2 = vacayspots1.loc[vacayspots1['Wind Speed']<10]
vacayspots3 = vacayspots2.loc[(vacayspots2['Humidity']>60) & (vacayspots2['Humidity']<70) ]
vacayspots = vacayspots3.loc[vacayspots3['Cloudiness']<50] 
vacayspots

,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
390,Draguignan,FR,1630791462,43.5333,6.4667,71.80,68,13,4.61
392,Ribeira Grande,PT,1630791418,38.5167,-28.7000,70.20,68,40,8.05
398,Mt. Pleasant,US,1630791416,32.7941,-79.8626,84.97,68,1,7.00
403,Kloulklubed,PW,1630791419,7.0419,134.2556,75.70,67,29,5.84
404,Safi,MA,1630791523,32.2994,-9.2372,73.71,67,12,8.39
410,Hilo,US,1630790958,19.7297,-155.0900,82.27,66,40,9.22
414,Paphos,CY,1630791243,34.7667,32.4167,80.51,65,0,6.91
417,Kiryat Gat,IL,1630791542,31.6100,34.7642,81.28,65,0,5.06
421,Broome,US,1630791515,42.2506,-75.8330,76.55,64,3,1.01
428,Benghazi,LY,1630791332,32.1167,20.0667,78.49,63,0,8.21


In [6]:
vacaylocay = vacayspots[["Latitude", "Longitude"]]
vacayhum = vacayspots["Humidity"].astype(float)


mapa2 = gmaps.figure()


vlaga2 = gmaps.heatmap_layer(vacaylocay, weights=vacayhum, dissipating=False, max_intensity=100, 
                             point_radius=2)


mapa2.add_layer(vlaga2)

mapa2

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = vacayspots[['City', 'Country', 'Latitude', 'Longitude']]
hotel_df['Hotel Name'] = ""
hotel_df

<ipython-input-16-617cba52e911>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Latitude,Longitude,Hotel Name
390,Draguignan,FR,43.5333,6.4667,
392,Ribeira Grande,PT,38.5167,-28.7000,
398,Mt. Pleasant,US,32.7941,-79.8626,
403,Kloulklubed,PW,7.0419,134.2556,
404,Safi,MA,32.2994,-9.2372,
410,Hilo,US,19.7297,-155.0900,
414,Paphos,CY,34.7667,32.4167,
417,Kiryat Gat,IL,31.6100,34.7642,
421,Broome,US,42.2506,-75.8330,
428,Benghazi,LY,32.1167,20.0667,


In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [17]:

for index, row in hotel_df.iterrows():

    lat = row["Latitude"]
    long = row["Longitude"]
    city_name = row["City"]
    

    params["location"] = f"{lat},{long}"


    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    

    results = response['results']
    

    try:
        print(f"Hotel in {city_name} called {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']


    except (KeyError, IndexError):
        print("This city isn't for vacationing")
        
    print("------------")
    

print("-------End of Search-------")

Retrieving Results for Index 390: Draguignan.
Hotel in Draguignan called Hôtel du Parc.
------------
Retrieving Results for Index 392: Ribeira Grande.


C:\Users\Dan\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Hotel in Ribeira Grande called Azul Singular.
------------
Retrieving Results for Index 398: Mt. Pleasant.
Hotel in Mt. Pleasant called Hotel Indigo Mount Pleasant, an IHG Hotel.
------------
Retrieving Results for Index 403: Kloulklubed.
Hotel in Kloulklubed called The Adventures Inn-Peleliu.
------------
Retrieving Results for Index 404: Safi.
Hotel in Safi called Hôtel Atlantique panorma.
------------
Retrieving Results for Index 410: Hilo.
Hotel in Hilo called Grand Naniloa Hotel Hilo - a DoubleTree by Hilton.
------------
Retrieving Results for Index 414: Paphos.
Hotel in Paphos called Elysium Hotel.
------------
Retrieving Results for Index 417: Kiryat Gat.
Hotel in Kiryat Gat called Desert Gat.
------------
Retrieving Results for Index 421: Broome.
This city isn't for vacationing
------------
Retrieving Results for Index 428: Benghazi.
Hotel in Benghazi called Tibesti Hotel.
------------
Retrieving Results for Index 431: Freeport.
Hotel in Freeport called The Freeport Inn and Ma

In [18]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
390,Draguignan,FR,43.5333,6.4667,Hôtel du Parc
392,Ribeira Grande,PT,38.5167,-28.7000,Azul Singular
398,Mt. Pleasant,US,32.7941,-79.8626,"Hotel Indigo Mount Pleasant, an IHG Hotel"
403,Kloulklubed,PW,7.0419,134.2556,The Adventures Inn-Peleliu
404,Safi,MA,32.2994,-9.2372,Hôtel Atlantique panorma
410,Hilo,US,19.7297,-155.0900,Grand Naniloa Hotel Hilo - a DoubleTree by Hilton
414,Paphos,CY,34.7667,32.4167,Elysium Hotel
417,Kiryat Gat,IL,31.6100,34.7642,Desert Gat
421,Broome,US,42.2506,-75.8330,
428,Benghazi,LY,32.1167,20.0667,Tibesti Hotel


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [23]:

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

mapa2.add_layer(markers)


mapa2

Figure(layout=FigureLayout(height='420px'))